In [3]:
#Data Exploration
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
donations = pd.read_csv("Data/donations.csv")
outcomes = pd.read_csv("Data/outcomes.csv")
projects = pd.read_csv("Data/projects.csv")


In [4]:
donations_projects = projects.merge(donations,on='projectid',how="right")

In [5]:
projects.groupby("resource_type").size()
#Most projects are for supplies or technologies, followed by books as the next most frequent

resource_type
Books         142394
Other          69791
Supplies      229213
Technology    216145
Trips           5499
Visitors        1011
dtype: int64

In [6]:
#recode thoughtful donor variable for easier data exploration and useful for later model building
outcomes.loc[outcomes['donation_from_thoughtful_donor'] == 't', 'thoughtful_donor'] = 1
outcomes.loc[outcomes['donation_from_thoughtful_donor'] == 'f', 'thoughtful_donor'] = 0

In [7]:
#interested to see how many projects there are by poverty level
print(projects.groupby("poverty_level").size())
#there is a linear trend between higher poverty level and number of projects
projects_outcomes = projects.merge(outcomes,on='projectid')
#recode "is exciting" variable and 'fully_funded' variable
projects_outcomes.loc[projects_outcomes['is_exciting'] == 't', 'exciting'] = 1
projects_outcomes.loc[projects_outcomes['is_exciting'] == 'f', 'exciting'] = 0
projects_outcomes.loc[projects_outcomes['fully_funded'] == 't', 'ff_num'] = 1
projects_outcomes.loc[projects_outcomes['fully_funded'] == 'f', 'ff_num'] = 0
projects_outcomes.loc[projects_outcomes['school_zip'] != 0,'counter'] = 1
#how many projects are exciting by povery level
p_o_grouped = projects_outcomes.groupby('poverty_level').sum()
p_o_grouped["percent_exciting"] = p_o_grouped["exciting"]/p_o_grouped["counter"]
x = p_o_grouped["percent_exciting"]
print(x)
#there is not a linear trend between poverty level and number of exciting projects

poverty_level
high poverty        173561
highest poverty     383489
low poverty          16711
moderate poverty     90337
dtype: int64
poverty_level
high poverty        0.055447
highest poverty     0.060004
low poverty         0.062025
moderate poverty    0.063045
Name: percent_exciting, dtype: float64


In [8]:
import numpy as np
print(projects_outcomes['total_price_excluding_optional_support'].mean())
print(projects_outcomes['total_price_excluding_optional_support'].median())
print(projects_outcomes['total_price_excluding_optional_support'].std())
projects_outcomes['total_price_excluding_optional_support'].plot.hist(bins=np.linspace(0,1500))
plt.show()
#distribution of project prices is very right-skewed and the standard deviation is very high. 

535.9817510486903
410.66
13125.95802802399


In [9]:
p_o_grouped = projects_outcomes.groupby('poverty_level').sum()
p_o_grouped.head()
p_o_grouped["percent_funded"] = p_o_grouped["ff_num"]/p_o_grouped["counter"]
p_o_grouped["percent_funded"]

poverty_level
high poverty        0.658163
highest poverty     0.726354
low poverty         0.669308
moderate poverty    0.639796
Name: percent_funded, dtype: float64

In [10]:
#The highest poverty grouping has the highest amount of fully funded projects, which is a positive sign
#It would be concerning if projects in low poverty schools had a greater amount of funding
#At the same time, the fact that 66% of projects at low poverty schools are fully funded is questionable
#Are all of those projects really necessary or could some of that money be better spent on
#high poverty and highest poverty schools?

In [11]:
projects_by_state = projects.groupby("school_state").count()
projects_by_state['state'] = projects_by_state.index
print(projects_by_state['projectid'].min())
print(projects_by_state['projectid'].max())
#There is a huge range in the number of projects by state

3
126242


In [13]:
p_by_s_mean = projects.groupby("school_state").mean()
print(p_by_s_mean["total_price_excluding_optional_support"].mean())
print(p_by_s_mean['total_price_excluding_optional_support'].median())
print(p_by_s_mean['total_price_excluding_optional_support'].std())
#The means and medians by state are not significantly different from those
#by poverty level. However, there is far less variance in project costs by state
#as compared to project costs by poverty level. Standard deviation was over 13000 
#for mean project costs by povery level

512.6267612551935
500.3494735476628
59.70915887067716


In [14]:
#Look at what types of projects are exciting
exciting = projects_outcomes[projects_outcomes["exciting"]==1]
non_exciting = projects_outcomes[projects_outcomes["exciting"]!=1]
print(exciting['total_price_excluding_optional_support'].mean())
print(non_exciting['total_price_excluding_optional_support'].mean())
e = exciting.groupby("resource_type").size()
n = non_exciting.groupby("resource_type").size()
tot_e = 0
tot_n = 0
for cat in e:
    tot_e += cat
for cat in n:
    tot_n += cat
header = ["Books","Other","Supplies","Tech","Trips","Visitors"]
lists = []
l1 = []
l2 = []
for i in e:
    l1.append(i/tot_e)
for i in n:
    l2.append(i/tot_n)
lists.append(l1)
lists.append(l2)
category_comp = pd.DataFrame(lists,columns=header)
c = category_comp.rename(index={0: 'Exciting',1:'Non-exciting'})
c.T.plot.bar(rot=0)
plt.title("Project Type Comparison between Exciting and Non-exciting")
plt.show()

#look at what types of schools are using Donors.org
#Exciting projects are on average less expensive than nonexciting projects, which is interesting
projects_outcomes.loc[(projects_outcomes['school_charter'] == 't')|(projects_outcomes['school_kipp'] == 't')
                      |(projects_outcomes['school_magnet'] == 't')|
                      (projects_outcomes['school_charter_ready_promise'] == 't'),
                      'special'] = 1
projects_outcomes.loc[projects_outcomes["special"]!= 1,"special"] = 0
print(projects_outcomes.groupby("special").size())
#Given that most schools in the U.S. overall aren't magnet/charter/kipp, it is interesting that a 
#significant portion of projects are submitted by teachers at these types of schools



494.5225905747767
538.5940476230993
special
0.0    508129
1.0    111197
dtype: int64


In [ ]:
##similar exercise as excitig vs. non-exciting for fully-funded vs. not fully-funded
funded = projects_outcomes[projects_outcomes["fully_funded"]=='t']
non_ff = projects_outcomes[projects_outcomes["fully_funded"]=='f']
f = funded.groupby("resource_type").size()
nonf = non_ff.groupby("resource_type").size()
tot_f = 0
tot_nonf = 0
totals = []
for cat in projects_outcomes.groupby("resource_type").size():
    totals.append(cat)    
#for cat in f:
#    tot_f += cat
#for cat in nonf:
#    tot_nonf += cat
header = ["Books","Other","Supplies","Tech","Trips","Visitors"]
lists = []
l1 = []
l2 = []
counter1 = 0
counter2 = 0
for i in f:
    l1.append(i/totals[counter1])
    counter1 += 1
for i in nonf:
    l2.append(i/totals[counter2])
    counter2 += 1
lists.append(l1)
lists.append(l2)
category_comp = pd.DataFrame(lists,columns=header)
c = category_comp.rename(index={0: 'Fully Funded',1:'Not Fully Funded'})
c.T.plot.bar(rot=0)
plt.title("Project Type Comparison between Fully Funded and Not")
plt.show()

In [15]:
#Data Story
#Something very interesting that this data portrays, is how donations serve different needs 
#for different types of schools. This is the reason that I have done a lot of exploration based
#on socioeconomic status. While the differences are not dramatic, they are still significant.
#First, you see that different types of schools request different types of resources.
c = pd.crosstab(index=projects["poverty_level"],columns=projects["resource_type"]).apply(lambda r: r/r.sum(), axis=1)
print(c)
#Teachers in high poverty schools request more books, while teachers in low poverty schools request more technology

resource_type        Books     Other  Supplies  Technology     Trips  Visitors
poverty_level                                                                 
high poverty      0.207837  0.105350  0.333896    0.345714  0.005837  0.001366
highest poverty   0.224823  0.105755  0.359766    0.297595  0.010389  0.001672
low poverty       0.182226  0.092101  0.298983    0.418971  0.006344  0.001376
moderate poverty  0.188947  0.104232  0.313428    0.387791  0.004384  0.001218


In [16]:
hmm = projects_outcomes.groupby('poverty_level').mean()['total_price_excluding_optional_support']
ax = hmm.plot(kind='bar',rot=0)
#ax.set_xlabel("Hour",fontsize=8,rotation=45)
ax.set_ylabel("Mean Price",fontsize=12)
plt.show()
twoway = projects_outcomes.groupby(['poverty_level','resource_type']).mean()['total_price_excluding_optional_support']
print(twoway)
#The distribution of project costs by poverty level is right-skewed. 
#The standard deviation of proejcts costs is very high
#The highest mean price for projects is the one for low poverty schools, which is interesting
#This may suggest that low poverty schools do not need donations for more basic supplies
#that cost less but high poverty schools still cannot get

poverty_level     resource_type
high poverty      Books             408.542445
                  Other             458.246673
                  Supplies          420.908364
                  Technology        664.141524
                  Trips            2381.275467
                  Visitors          964.756239
highest poverty   Books             418.234235
                  Other             745.711013
                  Supplies          418.205596
                  Technology        668.817141
                  Trips            2053.907476
                  Visitors         1107.885398
low poverty       Books             452.799062
                  Other             470.903593
                  Supplies          474.402223
                  Technology        753.769352
                  Trips            1858.700421
                  Visitors          988.301905
moderate poverty  Books             410.600515
                  Other             452.164373
                  Supplies  

In [17]:
#look at how donations vary by poverty level
donations_projects.groupby(["poverty_level"]).mean()
#Low poverty schools get the largest donations, unsurprisingly
#Interesting, there is very little variation in the amount of optional support given by poverty level
#which could present an opportunity for DonorsChose.org to get more money out of wealthier donors?

,school_ncesid,school_latitude,school_longitude,school_zip,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,donor_zip,donation_to_project,donation_optional_support,donation_total
poverty_level,,,,,,,,,,,,
high poverty,2.473408e+11,36.958640,-97.818668,59524.309255,29.661959,628.753862,743.592836,94.067739,53337.151860,62.834729,10.184012,73.018636
highest poverty,2.476224e+11,37.293905,-93.005818,52007.505186,28.770371,721.519528,858.625001,94.313612,48579.223100,67.945356,11.093919,79.038613
low poverty,1.875562e+11,36.923981,-100.259505,61935.411583,30.230401,648.122112,765.585965,91.623871,57729.434035,74.400670,12.163064,86.563733
moderate poverty,2.575351e+11,37.527498,-96.306059,56664.875749,30.168340,606.401602,715.864219,101.215617,53600.806668,62.040868,10.011141,72.052720


In [ ]:
'''Questions for Project Partners
1. It would be interesting to compare the degree to which donations are vital, for example
by having an algorithm that analyzes the text to determine how important a project is 
for the success of the students in the classroom
2. More demographic data would be interesting for understanding what types of students
and schools these donations are serving, and help DonorsChoose.org really quantify their impact
3. It is interesting that there are so many projects but only 15 designated "thoughtful donors"
I think that the data could be explored further in order to understand what makes a trustworthy donor
and create a way of identifying more donors who are making an impact'''

In [ ]:
"""Model to predict how much funding a project will receive
This is relevant to DonorsChoose.org because if it can identify projects that are both
"exciting/important" and WILL NOT receive sufficient funding, then they can do more to promote
those projects on their website, helping schools in need get the support that they need

Logistic regression with outcome variable = fully funded
Predictor variables: poverty level, resource type, 
project_cost, number of projects @ that school, whether or not school is charter/magnet
"""  


In [19]:
import statsmodels.api as sm
import pylab as pl
import numpy as np
import pandas as pd

#count number of projects by school
counts = projects.groupby("schoolid").count().reset_index()
num_per_school = counts[["schoolid","projectid"]]
num_per_school.columns = ['schoolid', 'num_projects']

#merge in data
model = projects_outcomes.merge(num_per_school,on='schoolid',how='left')

#generate dummy variables for categorical data
dummies1 = pd.get_dummies(model["poverty_level"],prefix="plevel")
dummies2 = pd.get_dummies(model["resource_type"],prefix="rtype")
model = model[['total_price_excluding_optional_support','num_projects','special','ff_num']]
cleaned = model.join(dummies1)
cleaned = cleaned.join(dummies2)

#seperate into training and testing
msk = np.random.rand(len(model)) < 0.8
train = cleaned[msk]
test = cleaned[~msk]
#X variables
predictors = train.drop(['ff_num'],axis=1)
test_predictors = test.drop(['ff_num'],axis=1)
#Construct model
logit = sm.Logit(train['ff_num'],predictors.astype(float))
result = logit.fit()
#summary results and odds ratios
print(result.summary())
print(np.exp(result.params))
y_pred = result.predict(test_predictors)
test["prediction"] = y_pred
ff = test[test["ff_num"]==1]
print(ff["prediction"].mean())
nff = test[test["ff_num"]==0]
print(nff["prediction"].mean())
#From this exercise, especially by comparing the odds-ratios, we can learn what types of projects
#have a higher probability of being funded, or, conversely, a higher probability of NOT being funded
#Trips and books have the highest likelihood of being funded, while visitors have a lower probability of being funded
#Some of the trends are what we saw in the data exploration, but there are also less obvious factors
#The number of projects at the project's school has a positive impact on receiving funding
#This suggests that projects from schools with fewer users of DonorsChoose.org might receive less funding
#despite having projects of equal quality, and this is something that DonorsChoose.org could address
#this is a very basic model. It could be improved with more time to include additional variables,
#interactions of variables, natural language processing of the project descriptions, or more advanced
#machine learning techniques

Optimization terminated successfully.
         Current function value: 0.591083
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                 ff_num   No. Observations:               494844
Model:                          Logit   Df Residuals:                   494831
Method:                           MLE   Df Model:                           12
Date:                Fri, 21 Oct 2016   Pseudo R-squ.:                 0.03810
Time:                        14:01:30   Log-Likelihood:            -2.9249e+05
converged:                       True   LL-Null:                   -3.0408e+05
                                        LLR p-value:                     0.000
                                             coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------------------------
total_price_excluding_optional_support    -0.0007   

/Users/SherryShenker/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
